In [21]:
import cv2
import numpy as np

In [2]:
#进行Contour_extraction函数进行轮廓提取

def Contour_extraction(img):

    img = cv2.GaussianBlur(img, (3, 3), 3)
    th = cv2.adaptiveThreshold(img, 250, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY, 35, 11)
    th = cv2.bitwise_not(th)
    kernel = np.array([[0, 1, 1],
                           [0, 1, 0],
                           [1, 1, 0]], dtype='uint8')
    th = cv2.morphologyEx(th, cv2.MORPH_CLOSE, kernel)
    return th


   template_path = r"E:\python files\PycharmProjects\CompleteProject\4.png"
    #读取灰度图像
    template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
    # 显示归一化后的结果
    cv2.imshow("模板字", template )
    cv2.waitKey(0)
    cv2.destroyAllWindows()


IndentationError: unindent does not match any outer indentation level (<string>, line 16)

In [22]:
def calculate_black_pixels(image):
    """ 计算图像中黑色像素的数量 """
    return np.sum(image == 0)

def get_center_of_mass(image):
    """ 计算图像的重心坐标 """
    black_pixels = np.argwhere(image == 0)
    if len(black_pixels) == 0:
        return image.shape[1] // 2, image.shape[0] // 2  # 返回图像中心
    center_y, center_x = np.mean(black_pixels, axis=0).astype(int)
    return center_x, center_y

def resize_and_center_image(template, student, output_size=(400, 400)):
    # Step 1: 计算两个图像中的黑色像素数量
    S_T = calculate_black_pixels(template)
    S_C = calculate_black_pixels(student)
    
    # Step 2: 提取模板和临摹字的ROI区域
    x_T, y_T, w_T, h_T = cv2.boundingRect(template)
    template_roi = template[y_T:y_T+h_T, x_T:x_T+w_T]
    
    x_C, y_C, w_C, h_C = cv2.boundingRect(student)
    student_roi = student[y_C:y_C+h_C, x_C:x_C+w_C]
    
    # Step 3: 计算缩放因子并调整临摹字的大小
    scaling_factor = np.sqrt(S_T / S_C)
    new_width = int(w_C * scaling_factor)
    new_height = int(h_C * scaling_factor)
    resized_student = cv2.resize(student_roi, (new_width, new_height))
    
    # Step 4: 创建400x400的空白背景
    blank_template = np.ones(output_size, dtype=np.uint8) * 255  # 模板字空白图
    blank_student = np.ones(output_size, dtype=np.uint8) * 255   # 临摹字空白图
    
    # Step 5: 获取两个图像的重心
    template_center_x, template_center_y = get_center_of_mass(template_roi)
    student_center_x, student_center_y = get_center_of_mass(resized_student)
    
    # Step 6: 将模板字放到空白图的中心
    start_x_T = output_size[1] // 2 - w_T // 2
    start_y_T = output_size[0] // 2 - h_T // 2
    blank_template[start_y_T:start_y_T+h_T, start_x_T:start_x_T+w_T] = template_roi
    
    # Step 7: 将调整后的临摹字放到空白图的中心
    start_x_C = output_size[1] // 2 - resized_student.shape[1] // 2
    start_y_C = output_size[0] // 2 - resized_student.shape[0] // 2
    blank_student[start_y_C:start_y_C+resized_student.shape[0], start_x_C:start_x_C+resized_student.shape[1]] = resized_student
    
    return blank_template, blank_student

In [23]:
# 主函数用于加载图像并处理
def main():
    template_path = r"E:\python files\PycharmProjects\CompleteProject\4.png"
    student_path = r"E:\python files\PycharmProjects\CompleteProject\18.png"
    
    # 读取灰度图像
    template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
    student = cv2.imread(student_path, cv2.IMREAD_GRAYSCALE)
    
    if template is None or student is None:
        print("Error loading images.")
        return
    
    # 调整临摹字大小并将它们都显示在400x400的空白图中
    aligned_template, aligned_student = resize_and_center_image(template, student)
    
    # 显示归一化后的结果
    cv2.imshow("模板字", aligned_template)
    cv2.imshow("临摹字", aligned_student)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()